In [2]:
import numpy as np
import qutip as qt
import matplotlib.pyplot as plt
from ipywidgets import interact, SelectionSlider
from mytoolbox import odmr
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

# Constants
D = 2.87  # Zero-field splitting in GHz
g_e = -28.03  # Gyromagnetic ratio of the electron in GHz/T
t = 50  # Time for evolution in arbitrary units
n_evolution = 200  # Number of data points for numerical solution
initial_state = qt.basis(3, 1)
evolution_operator = qt.jmat(1, 'x')  # Using Sx as evolution operator

# Frequency range
frequencies = np.linspace(D - 1, D + 1, 1000)

# NV orientations
nv_orientations = [
    (1, 1, 1),
    (1, -1, -1),
    (-1, 1, -1),
    (-1, -1, 1)
]

# Precompute fidelities for all combinations
B_values = [0.0001, 0.005, 0.01, 0.015]
polar_values = [0, 1, 5, 10, 20, 45, 54.7356]
azimuthal_values = [0, 1, 5, 10, 20, 44, 45, 225]
amp_values = [0.01, 0.05]

precomputed_fidelities = {}

start = time.time()

for B in B_values:
    print(f"Computing B={B}")
    for polar in polar_values:
        for azimuthal in azimuthal_values:
            for amp in amp_values:
                avg_fidelity = np.zeros(len(frequencies))
                individual_fidelities = []
                for orientation in nv_orientations:
                    fidelities = odmr(D, g_e, B, polar, azimuthal, orientation, frequencies, t, n_evolution, initial_state, evolution_operator, amp)
                    individual_fidelities.append(fidelities)
                    avg_fidelity += np.array(fidelities)
                avg_fidelity /= len(nv_orientations)
                precomputed_fidelities[(B, polar, azimuthal, amp)] = (individual_fidelities, avg_fidelity)
        print(f'Computed all values for polar angle {polar} and B {B}, time elapsed {time.time() - start}')

# precomputed_fidelities = {}

# # Start time
# start = time.time()

# # Function to compute fidelities for a given set of parameters
# def compute_fidelities(B, polar, azimuthal, amp):
#     avg_fidelity = np.zeros(len(frequencies))
#     individual_fidelities = []
#     for orientation in nv_orientations:
#         fidelities = odmr(D, g_e, B, polar, azimuthal, orientation, frequencies, t, n_evolution, initial_state, evolution_operator, amp)
#         individual_fidelities.append(fidelities)
#         avg_fidelity += np.array(fidelities)
#     avg_fidelity /= len(nv_orientations)
#     return (B, polar, azimuthal, amp), (individual_fidelities, avg_fidelity)

# # Create a ThreadPoolExecutor
# with ThreadPoolExecutor(max_workers=8) as executor:
#     # List to store futures
#     futures = []

#     # Submit tasks to the executor
#     for B in B_values:
#         for polar in polar_values:
#             for azimuthal in azimuthal_values:
#                 for amp in amp_values:
#                     futures.append(executor.submit(compute_fidelities, B, polar, azimuthal, amp))

#     # Process completed futures as they finish
#     for future in as_completed(futures):
#         params, result = future.result()
#         precomputed_fidelities[params] = result

#         # Print progress
#         B, polar, azimuthal, amp = params
#         print(f'Computed values for B={B}, polar={polar}, azimuthal={azimuthal}, amp={amp}, time elapsed {time.time() - start}')

# print(f'Total computation time: {time.time() - start} seconds')

def plot_fidelities(B, polar, azimuthal, amp):
    try:
        individual_fidelities, avg_fidelity = precomputed_fidelities[(B, polar, azimuthal, amp)]
    except KeyError:
        print(f"Combination (B={B}, polar={polar}, azimuthal={azimuthal}, amp={amp}) not precomputed.")
        return
    
    plt.figure(figsize=(14, 6))

    # Plot individual fidelities
    plt.subplot(1, 2, 1)
    for i, fid in enumerate(individual_fidelities):
        plt.plot(frequencies, fid, label=f'Orientation {i+1}')
    plt.xlabel('Frequency (GHz)')
    plt.ylabel('Fidelity')
    plt.title('ODMR Signal for Each NV Orientation')
    plt.legend()
    plt.grid(True)

    # Plot average fidelity
    plt.subplot(1, 2, 2)
    plt.plot(frequencies, avg_fidelity, label='Average Fidelity')
    plt.xlabel('Frequency (GHz)')
    plt.ylabel('Fidelity')
    plt.title('Average ODMR Signal')
    plt.legend()
    plt.grid(True)

    plt.show()

B_slider = SelectionSlider(options=B_values, value=0.015, description='B (T)')
polar_slider = SelectionSlider(options=polar_values, value=0, description='Polar (°)')
azimuthal_slider = SelectionSlider(options=azimuthal_values, value=0, description='Azimuthal (°)')
amp_slider = SelectionSlider(options=amp_values, value=0.05, description='Amplitude')

# Create interactive plot
interact(plot_fidelities, B=B_slider, polar=polar_slider, azimuthal=azimuthal_slider, amp=amp_slider)


Computing B=0.0001
Computed all values for polar angle 0 and B 0.0001, time elapsed 447.99070143699646
Computed all values for polar angle 1 and B 0.0001, time elapsed 905.6497178077698
Computed all values for polar angle 5 and B 0.0001, time elapsed 1339.0148782730103
Computed all values for polar angle 10 and B 0.0001, time elapsed 1761.1348967552185
Computed all values for polar angle 20 and B 0.0001, time elapsed 2177.057571411133
Computed all values for polar angle 45 and B 0.0001, time elapsed 2590.64195394516
Computed all values for polar angle 54.7356 and B 0.0001, time elapsed 3002.711014032364
Computing B=0.005
Computed all values for polar angle 0 and B 0.005, time elapsed 3426.328943967819
Computed all values for polar angle 1 and B 0.005, time elapsed 3849.2887918949127
Computed all values for polar angle 5 and B 0.005, time elapsed 4273.423166036606
Computed all values for polar angle 10 and B 0.005, time elapsed 4695.997659921646
Computed all values for polar angle 20 an

interactive(children=(SelectionSlider(description='B (T)', index=3, options=(0.0001, 0.005, 0.01, 0.015), valu…

<function __main__.plot_fidelities(B, polar, azimuthal, amp)>

In [3]:
import os
# Ensure the directory for saving plots exists
output_folder = 'slider_plots'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)


for B in B_values:
    print(f"Computing B={B}")
    for polar in polar_values:
        for azimuthal in azimuthal_values:
            for amp in amp_values:
                avg_fidelity = np.zeros(len(frequencies))
                individual_fidelities = []
                
                # Compute fidelities for each NV orientation
                for orientation in nv_orientations:
                    fidelities = odmr(D, g_e, B, polar, azimuthal, orientation, frequencies, t, n_evolution, initial_state, evolution_operator, amp)
                    individual_fidelities.append(fidelities)
                    avg_fidelity += np.array(fidelities)
                
                avg_fidelity /= len(nv_orientations)
                
                # Create plots
                fig, axes = plt.subplots(1, len(nv_orientations) + 1, figsize=(20, 5))
                for i, fidelities in enumerate(individual_fidelities):
                    axes[i].plot(frequencies, fidelities)
                    axes[i].set_title(f'Orientation {nv_orientations[i]}')
                    axes[i].set_xlabel('Frequency (GHz)')
                    axes[i].set_ylabel('Fidelity')
                
                # Plot the average fidelity
                axes[-1].plot(frequencies, avg_fidelity, label='Average Fidelity')
                axes[-1].set_title('Average Fidelity')
                axes[-1].set_xlabel('Frequency (GHz)')
                axes[-1].set_ylabel('Fidelity')
                
                # Set the overall title
                fig.suptitle(f'B={B}, Polar={polar}, Azimuthal={azimuthal}, Amplitude={amp}', fontsize=16)
                
                # Save the plot
                filename = f'{output_folder}/B_{B}_Polar_{polar}_Azimuthal_{azimuthal}_Amp_{amp}.png'.replace('.', 'p')
                plt.savefig(filename)
                plt.close(fig)
                print(f"Saved plot: {filename}")

Computing B=0.0001
Saved plot: slider_plots/B_0p0001_Polar_0_Azimuthal_0_Amp_0p01ppng
Saved plot: slider_plots/B_0p0001_Polar_0_Azimuthal_0_Amp_0p05ppng
Saved plot: slider_plots/B_0p0001_Polar_0_Azimuthal_1_Amp_0p01ppng
Saved plot: slider_plots/B_0p0001_Polar_0_Azimuthal_1_Amp_0p05ppng
Saved plot: slider_plots/B_0p0001_Polar_0_Azimuthal_5_Amp_0p01ppng
Saved plot: slider_plots/B_0p0001_Polar_0_Azimuthal_5_Amp_0p05ppng
Saved plot: slider_plots/B_0p0001_Polar_0_Azimuthal_10_Amp_0p01ppng
Saved plot: slider_plots/B_0p0001_Polar_0_Azimuthal_10_Amp_0p05ppng
Saved plot: slider_plots/B_0p0001_Polar_0_Azimuthal_20_Amp_0p01ppng
Saved plot: slider_plots/B_0p0001_Polar_0_Azimuthal_20_Amp_0p05ppng
Saved plot: slider_plots/B_0p0001_Polar_0_Azimuthal_44_Amp_0p01ppng
Saved plot: slider_plots/B_0p0001_Polar_0_Azimuthal_44_Amp_0p05ppng
Saved plot: slider_plots/B_0p0001_Polar_0_Azimuthal_45_Amp_0p01ppng
Saved plot: slider_plots/B_0p0001_Polar_0_Azimuthal_45_Amp_0p05ppng
Saved plot: slider_plots/B_0p0001_P

In [4]:
def plot_fidelities(B, polar, azimuthal, amp):
    try:
        individual_fidelities, avg_fidelity = precomputed_fidelities[(B, polar, azimuthal, amp)]
    except KeyError:
        print(f"Combination (B={B}, polar={polar}, azimuthal={azimuthal}, amp={amp}) not precomputed.")
        return
    
    plt.figure(figsize=(14, 6))

    # Plot individual fidelities
    plt.subplot(1, 2, 1)
    for i, fid in enumerate(individual_fidelities):
        plt.scatter(frequencies, fid, label=f'Orientation {i+1}')
    plt.xlabel('Frequency (GHz)')
    plt.ylabel('Fidelity')
    plt.axvline(2.87, color='grey')
    plt.title('ODMR Signal for Each NV Orientation')
    plt.legend()
    plt.grid(True)

    # Plot average fidelity
    plt.subplot(1, 2, 2)
    plt.scatter(frequencies, avg_fidelity, label='Average Fidelity')
    plt.xlabel('Frequency (GHz)')
    plt.ylabel('Fidelity')
    plt.title('Average ODMR Signal')
    plt.legend()
    plt.grid(True)

    plt.show()

B_slider = SelectionSlider(options=B_values, value=0.015, description='B (T)')
polar_slider = SelectionSlider(options=polar_values, value=0, description='Polar (°)')
azimuthal_slider = SelectionSlider(options=azimuthal_values, value=0, description='Azimuthal (°)')
amp_slider = SelectionSlider(options=amp_values, value=0.05, description='Amplitude')

# Create interactive plot
interact(plot_fidelities, B=B_slider, polar=polar_slider, azimuthal=azimuthal_slider, amp=amp_slider)

interactive(children=(SelectionSlider(description='B (T)', index=3, options=(0.0001, 0.005, 0.01, 0.015), valu…

<function __main__.plot_fidelities(B, polar, azimuthal, amp)>